In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F

### Inception Model 블록 구조

In [2]:
class InceptionModule(nn.Module) : 
    
    def __init__(self, in_channels, out_1x1, reduce_3x3, out_3x3, reduce_5x5, out_5x5, out_pool) : 
        super(InceptionModule, self).__init__()
        
        # 1x1 convolution branch
        self.conv1x1 = nn.Conv2d(in_channels, out_1x1, kernel_size=1)
        # ---->>>> 입력 in_channels ->  out_1x1 변환 
        # 3x3 convolution branch
        self.conv3x3_reduce = nn.Conv2d(in_channels, reduce_3x3, kernel_size=1) # reduce : 줄이다.. 
        # ---->>>> (목적 : 입력 채널 줄이기) -> 입력 in_channels -> reduce_3x3 변환 
        self.conv3x3 = nn.Conv2d(reduce_3x3, out_3x3, kernel_size=3, padding=1)
        # ---->>>> 입력 : reduce_3x3 -> out_3x3 변환 / 출력 크기를 입력과 동일하게 유지 합니다. 

        # 5x5 convolution branch
        self.conv5x5_reduce = nn.Conv2d(in_channels, reduce_5x5, kernel_size=1)
        self.conv5x5 = nn.Conv2d(reduce_5x5, out_5x5, kernel_size=5, padding=2)
        
        # Max Pooling branch 
        self.pool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.conv_pool = nn.Conv2d(in_channels, out_pool, kernel_size=1)
        
    def forward(self, x) : 
        # 1x1 convolution branch
        out_1x1 = F.relu(self.conv1x1(x))
        
        # 3x3 convolution branch
        out_3x3 = F.relu(self.conv3x3(F.relu(self.conv3x3_reduce(x))))
        ### 순서 : self.conv3x3_reduce(x) -> relu -> conv3x3(x) -> relu
        
        # 5x5 convolution branch
        out_5x5 = F.relu(self.conv5x5(F.relu(self.conv5x5_reduce(x))))
        ### 순서 : self.conv5x5_reduce(x) -> relu -> self.conv5x5(x) -> relu
        
        # Max Pooling branch 
        out_pool = F.relu(self.conv_pool(self.pool(x)))
        ### 순서 : self.pool(x) -> self.conv_pool(self.pool(x)) -> relu
        
        # Concatenate all branch 
        out = torch.cat([out_1x1, out_3x3, out_5x5, out_pool], dim=1)
        
        return out

### Inception 모델 정의

In [3]:
class InceptionV2(nn.Module) : 
    def __init__(self, num_classes=1000) : 
        super(InceptionV2, self).__init__()
        
        #  Stem layers ----> 모델 초기 단계에서 입력 이미지에 대해서 초기 특성 추출하는곳 
        self.stem = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            ### padding 3 -> 이미지에 가장자리에 대한 정보 손실 방지 
            nn.ReLU(inplace=True)
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
            ### padding 1 -> 입력값 가장자리에 대한 정보 손실 방지 
        )
        # Inception modules
        
        # Inception modules(in_channels, out_1x1, reduce_3x3, out_3x3, reduce_5x5, out_5x5, out_pool)        
        self.inception3a = InceptionModule(64, 64, 96, 128, 16, 32, 32)
        self.inception3b = InceptionModule(256, 128, 128, 192, 32, 96, 64)
        self.inception4a = InceptionModule(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = InceptionModule(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionModule(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionModule(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionModule(528, 256, 160, 320, 32, 128, 128)
        self.inception5a = InceptionModule(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionModule(832, 384, 192, 384, 48, 128, 128)
        
        
        # avg pool 
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(1024, num_classes)
        
    def forward(self, x) : 
        
        # Stem layers 
        x = self.stem(x)
        
        # Inception modules
        x = inception3a(x)
        x = inception3b(x)
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = inception4a(x)
        x = inception4b(x)
        x = inception4c(x)
        x = inception4d(x)
        x = inception4e(x)
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = inception5a(x)
        x = inception5b(x)
        
        # avg pool 
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x

SyntaxError: invalid syntax (3179068677.py, line 10)